In [1]:
import pandas as pd
import re
import nltk
import numpy as np

from nltk.corpus import sentiwordnet as swn

In [7]:
# 처음에 nltk 가 import 안될 경우는 패키지가 제대로 다운이 되지 않은 것 때문이므로 아래 명령어로 다운로드
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [23]:
## 나눠서 수집한 기사 결합
상폐기업뉴스1 = pd.read_csv('./datasets/상폐기업번역파일.csv')
상폐기업뉴스1.drop('뉴스기사본문전처리', axis=1, inplace=True)

상폐기업뉴스2 = pd.read_csv('./datasets/상폐기업번역파일추가.csv', encoding='utf-8', index_col=0)
상폐기업뉴스2.drop('뉴스기사본문전처리', axis=1, inplace=True)
상폐기업뉴스2.columns = ['기업', '기사발행일', '기사제목', '기사본문번역']

상폐기업뉴스 = pd.concat([상폐기업뉴스1, 상폐기업뉴스2], axis=0)
상폐기업뉴스.reset_index(inplace=True, drop=True)
상폐기업뉴스.drop('기사발행일', inplace=True, axis=1)

## 파파고가 번역할 때 생략된 한글 제거
상폐기업뉴스['기사본문번역'] = [re.sub('[가-힣]', '', s) for s in 상폐기업뉴스['기사본문번역']]


In [25]:
## 기사 토큰화 및 품사 태깅
상폐기업뉴스['기사본문번역토큰'] = 상폐기업뉴스['기사본문번역'].apply(lambda x: nltk.word_tokenize(x))
상폐기업뉴스['기사본문번역토큰'] = 상폐기업뉴스['기사본문번역토큰'].apply(lambda x: nltk.pos_tag(x))

In [26]:
## 문장에서 명사 동사 형용사 부사에 한하여 부정 긍정지수 계산
def word_sentiment_calculator(word, tag):
    pos_score = 0
    neg_score = 0
    
    if 'NN' in tag and len(list(swn.senti_synsets(word, 'n')))>0:
        syn_set = list(swn.senti_synsets(word, 'n'))
    elif 'VB' in tag and len(list(swn.senti_synsets(word, 'v')))>0:
        syn_set = list(swn.senti_synsets(word, 'v'))
    elif 'JJ' in tag and len(list(swn.senti_synsets(word, 'a')))>0:
        syn_set = list(swn.senti_synsets(word, 'a'))
    elif 'RB' in tag and len(list(swn.senti_synsets(word, 'r')))>0:
        syn_set = list(swn.senti_synsets(word, 'r'))
    else:
        return (0,0)
    
    for syn in syn_set:
        pos_score += syn.pos_score()
        neg_score += syn.neg_score()
    return (pos_score/len(syn_set), neg_score/len(syn_set))

## 출처 : https://github.com/buomsoo-kim/Sentiment-Analysis-with-Python/blob/master/source%20code/session%204%20-%20Text%20analysis/iphynb%20files/%5B4-1%5D%20SentiwordNet.ipynb

In [1]:
## 문장의 감성 지수 계산하기 위한 함수
def sentence_sentiment_calculator(sent):
    tokens =  nltk.word_tokenize(sent)
    pos_tags = nltk.pos_tag(tokens)
    
    pos_score = 0
    neg_score = 0
    for word, tag in pos_tags:
        pos_score += word_sentiment_calculator(word, tag)[0]
        neg_score += word_sentiment_calculator(word, tag)[1]
    return (pos_score, neg_score)

## 출처 : https://github.com/buomsoo-kim/Sentiment-Analysis-with-Python/blob/master/source%20code/session%204%20-%20Text%20analysis/iphynb%20files/%5B4-1%5D%20SentiwordNet.ipynb

In [28]:
## 긍정일경우 0 부정일경우 1
predicted = []

for i in range(len(상폐기업뉴스)):
    scores = sentence_sentiment_calculator(상폐기업뉴스.기사본문번역[i])
    
    #긍정점수 >= 부정점수 일 경우 긍정
    if scores[0] >= scores[1]:
        predicted.append(0)
    #부정점수 < 긍정점수 일 경우 부정
    else:
        predicted.append(1)

상폐기업뉴스['긍정부정분류'] = predicted

In [29]:
## Groupby활용하여 기업별 부정기사수, 전체기사수를 추출하고 부정기사비율로 계산
전체기사수 = pd.DataFrame(상폐기업뉴스.groupby(['기업']).count()['긍정부정분류'])
전체기사수.reset_index(drop = False, inplace = True)
전체기사수.columns = ['기업', '전체기사수']

부정기사수 = pd.DataFrame(상폐기업뉴스[상폐기업뉴스.긍정부정분류==1].groupby(['기업']).count()['긍정부정분류'])
부정기사수.reset_index(drop = False, inplace = True)
부정기사수.columns = ['기업', '부정기사수']

상폐기업 = pd.merge(전체기사수, 부정기사수, on=['기업'], how='left')
상폐기업['부정기사비율'] = (상폐기업['부정기사수'] / 상폐기업['전체기사수']) * 100
상폐기업.fillna(0, inplace=True)
상폐기업.drop(['전체기사수', '부정기사수'], axis=1, inplace=True)
상폐기업.columns = ['회사명', '부정기사비율']
상폐기업

,회사명,부정기사비율
0,KCW,33.333333
1,글로웍스,26.923077
2,금강제강,0.000000
3,나노트로닉스,19.230769
4,나이스메탈,28.048780
...,...,...
128,한와이어리스,31.578947
129,해피드림,11.428571
130,핸디소프트,32.608696
131,휴먼텍코리아,33.333333


In [32]:
## 상폐기업 재무비율을 가져와서 부정기사비율과 결합하여 저장
상폐기업재무비율 = pd.read_csv('../Step1-2_재무비율_모델링/datasets/재무비율.csv')
상폐기업재무비율 = 상폐기업재무비율[상폐기업재무비율.부실기업여부==1]
상폐기업_감성분석 = pd.merge(상폐기업재무비율, 상폐기업, how='left', on=['회사명'])
상폐기업_감성분석.fillna(0, inplace=True)
상폐기업_감성분석.to_csv('../Step5-2_감성분석_모델링/datasets/상폐기업_부정기사비율_데이터.csv', index=None, encoding='utf-8-sig')
